Evaluate the models

# Stochastic Adversarial Training (StochAT)

### SoTA

vanila SGD: 
MNIST - 99%+ (most cnns), CIFAR10 - 93%+ (resnet18), 96%+ (wideresnet) 

MNIST:

adversarial attacks: 
l-inf @ eps = 80/255 @20 steps: TRADES - 96.07% - (4 layer cnn), MART 96.4%, MMA 95.5%, PGD - 96.01% - (4 layer cnn)

adversarial attacks:
l-2 @ eps = 32/255 (check): TRADES, MMA, PGD

CIFAR10:

adversarial attacks: 
l-inf @ eps = 8/255 @20 steps: 
TRADES 53-56% - (WRN-34-10), MART 57-58% (WRN-34-10), MMA 47%, PGD 48% - (WRN-32-10)// 49% - (WRN-34-10), Std - 0.03%
https://openreview.net/pdf?id=rklOg6EFwS (Table 4)

adversarial attacks: 
l-inf @ eps = 8/255 @20 steps: 
[ResNet10] TRADES 45.4%, MART 46.6%, MMA 37.26%, PGD 42.27%, Std 0.14%

Benign accuracies: TRADES 84.92%, MART 83.62%, MMA 84.36, PGD 87.14%, Std 95.8% [wideresnet]
https://openreview.net/pdf?id=Ms9zjhVB5R (Table 1)

adversarial attacks:
l-2 @ eps = 32/255 (check): TRADES, MART, MMA, PGD

TBD: CWinf attacks

## Pretrained models for comparison

download pretrained models and place in ../trainedmodels/MNIST or ../trainedmodels/CIFAR10 respectively

### TRADES :
https://github.com/yaodongyu/TRADES (MNIST: small cnn, CIFAR10: WideResNet34)
### MMA : 
https://github.com/BorealisAI/mma_training (MNIST: lenet5, CIFAR10: WideResNet28)
### MART :
 https://github.com/YisenWang/MART (CIFAR10: ResNet18 and WideResNet34)

## IMPORT LIBRARIES

In [1]:
import numpy as np
import pandas as pd
from torch import nn, optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision import transforms, datasets
from multiprocessing import cpu_count
from collections import OrderedDict
import matplotlib.pyplot as plt
import torch
import olympic
import sys
from typing import Union, Callable, Tuple
sys.path.append('../adversarial/')
sys.path.append('../architectures/')
from functional import boundary, iterated_fgsm, local_search, pgd, entropySmoothing
from ESGD_utils import *
import pickle
import time
import torch.backends.cudnn as cudnn
import argparse, math, random
import ESGD_optim
from trades import trades_loss

In [2]:
if torch.cuda.is_available():
    DEVICE = 'cuda'
else:
    DEVICE = 'cpu'

In [3]:
DEVICE

'cuda'

# LOAD DATA

In [4]:
#place data folders outside working directory

In [5]:
dataset = 'CIFAR10' # [MNIST, CIFAR10]
transform = transforms.Compose([
    transforms.ToTensor(),
])
bsz = 128
if dataset == 'MNIST':
    train = datasets.MNIST('../../data/MNIST', train=True, transform=transform, download=True)
    val = datasets.MNIST('../../data/MNIST', train=False, transform=transform, download=True)
elif dataset == 'CIFAR10':
    train = datasets.CIFAR10('../../data/CIFAR10', train=True, transform=transform, download=True)
    val = datasets.CIFAR10('../../data/CIFAR10', train=False, transform=transform, download=True)
    
train_loader = DataLoader(train, batch_size=128, num_workers=cpu_count(),drop_last=True)
val_loader = DataLoader(val, batch_size=128, num_workers=cpu_count(),drop_last=True)

Extracting ../../data/CIFAR10/cifar-10-python.tar.gz to ../../data/CIFAR10
Files already downloaded and verified


# INITIALIZE NETWORK

In [6]:
if dataset=='MNIST':
    from net_mnist import Net, NetSoft, model_cnn
    Net = model_cnn
    NetName = 'model_cnn'

In [12]:
if dataset=='CIFAR10':
    #[ResNet18,ResNet34,ResNet50,WideResNet]
    from resnet import ResNet18,ResNet34,ResNet50
    from wideresnet import WideResNet
    #Net = WideResNet
    #NetName = 'WideResNet'
    Net = ResNet18
    NetName = 'ResNet18'

In [13]:
Net

<function resnet.ResNet18()>

# RANDOM SEED 

In [14]:
seed = 42
torch.set_num_threads(2)
if DEVICE=='cuda':
    torch.cuda.set_device(-1)
    torch.cuda.manual_seed(seed)
    cudnn.benchmark = True
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# LOAD PRETRAINED OR TRAIN NEW MODELS:

In [15]:
TrainSGD = True
TrainESGD = True
TrainL2 = True
TrainLInf = True
TrainSAT2 = True
TrainSATInf = True
TrainTRADES = True
TrainTRADESInf = True
# tbd: add training modules for MART(inf only), MMA and MMAInf - always keep False
TrainMART = False 
TrainMMA = False
TrainMMAInf = False

# TRAIN NAIVE MODEL USING SGD

In [18]:
if TrainSGD:
    ## initialize model
    model_SGD = Net().to(DEVICE)
    ## training params
    lr = 0.001
    optimiser = optim.SGD(model_SGD.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()
    epochs = 10
    ## train model
    history_natural = olympic.fit(
        model_SGD,
        optimiser,
        loss_fn,
        dataloader=train_loader,
        epochs=epochs,
        metrics=['accuracy'],
        prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)),
        callbacks=[
            olympic.callbacks.Evaluate(val_loader),
            olympic.callbacks.ReduceLROnPlateau(patience=5)
        ]
    )
    ## verify validation accuracy
    print('final validation accuracy:')
    valscore = olympic.evaluate(model_SGD, val_loader, metrics=['accuracy'],
                     prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)))
    print(valscore)
    ## save model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_SGD_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_SGD,modelname)


Epoch 1:   0%|          | 0/390 [00:00<?, ?it/s]

Begin training...



Epoch 1:  32%|███▏      | 123/390 [00:07<00:14, 18.10it/s, loss=2, accuracy=0.328]   


Epoch 1:  63%|██████▎   | 246/390 [00:14<00:07, 18.09it/s, loss=1.93, accuracy=0.281]


Epoch 1:  95%|█████████▍| 369/390 [00:21<00:01, 18.07it/s, loss=1.77, accuracy=0.398]


...

Epoch 10:  42%|████▏     | 164/390 [00:09<00:12, 18.08it/s, loss=0.94, accuracy=0.648] 


Epoch 10:  73%|███████▎  | 284/390 [00:16<00:06, 17.61it/s, loss=0.94, accuracy=0.672]


Epoch 10: 100%|██████████| 390/390 [00:24<00:00, 15.78it/s, loss=0.951, accuracy=0.67, val_loss=1.12, val_accuracy=0.599]

Finished.
final validation accuracy:


{'val_accuracy': 0.5990584935897436}


# TRAIN MODEL USING ENTROPY SGD (ESGD)

In [19]:
def entropy_training(model, optimiser, loss_fn, x, y, epoch):
    
    model.train()
    y_pred = model(x)
    
    def helper():
        def feval():
            #x, y = Variable(x), Variable(y.squeeze())
            bsz = x.size(0)
            optimiser.zero_grad()
            yh = model(x)
            f = loss_fn.forward(yh, y)
            f.backward()

            yp = yh.argmax(axis=1)
            prec1 = 100*torch.sum(yp == y)//bsz
            err = 100.-prec1.item()

            return (f.data.item(), err)
        return feval

    loss, err = optimiser.step(helper(), model, loss_fn)
    loss = torch.tensor(loss)
    return loss, y_pred

In [20]:
if TrainESGD:
    ## initialize model
    model_ESGD = Net().to(DEVICE)
    ## training parameters
    lr = 0.01 
    l2 = 0.0 #l2 regularization
    L = 0    #langevin iterations
    gamma = 1e-4 
    scoping = 1e-3
    noise = 1e-4
    loss_fn = nn.CrossEntropyLoss()
    epochs = 5
    optimiser = ESGD_optim.EntropySGD(model_ESGD.parameters(),
            config = dict(lr=lr, momentum=0.9, nesterov=True, weight_decay=l2,
            L=L, eps=noise, g0=gamma, g1=scoping))
    ## train model
    history_natural = olympic.fit(
        model_ESGD,
        optimiser,
        loss_fn,
        dataloader=train_loader,
        epochs=epochs,
        metrics=['accuracy'],
        prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)),
        update_fn=entropy_training,
        callbacks=[
            olympic.callbacks.Evaluate(val_loader),
            olympic.callbacks.ReduceLROnPlateau(patience=5)
        ]
    )
    ## verify validation accuracy
    print('final validation accuracy:')
    valacc = olympic.evaluate(model_ESGD, val_loader, metrics=['accuracy'],
                         prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)))
    print(valacc['val_accuracy'])
    ## save trained model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_ESGD_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_ESGD,modelname)


Epoch 1:   0%|          | 0/390 [00:00<?, ?it/s]

Begin training...


/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)

Epoch 1:  31%|███▏      | 122/390 [00:09<00:19, 13.70it/s, loss=1.34, accuracy=0.531]


Epoch 1:  63%|██████▎   | 244/390 [00:18<00:10, 13.61it/s, loss=1.04, accuracy=0.633] 


Epoch 1:  94%|█████████▎| 365/390 [00:27<00:01, 13.76it/s, loss=0.791, accuracy=0.711]


...

Epoch 5:  67%|██████▋   | 262/390 [00:20<00:09, 13.46it/s, loss=0.263, accuracy=0.898]


Epoch 5:  98%|█████████▊| 383/390 [00:29<00:00, 13.42it/s, loss=0.152, accuracy=0.953]


Epoch 5: 100%|██████████| 390/390 [00:31<00:00, 12.27it/s, loss=0.214, accuracy=0.925, val_loss=0.95, val_accuracy=0.753]

Finished.
final validation accuracy:


0.7530048076923077


# TRAIN MODEL USING PGD SGD 

In [35]:
def infnorm(x):
    infn = torch.max(torch.abs(x.detach().cpu()))
    return infn

In [36]:
def adversarial_training(model, optimiser, loss_fn, x, y, epoch, adversary, k, step, eps, norm, random):
    """Performs a single update against a specified adversary"""
    model.train()

    # Adversial perturbation
    x_adv = adversary(model, x, y, loss_fn, k=k, step=step, eps=eps, norm=norm, random=True)
    #print('l2:',torch.norm(x_adv.detach().cpu()-x.detach().cpu())/np.sqrt(x.detach().cpu().size(0)))    
    #print('linf:',infnorm(x_adv.detach().cpu()-x.detach().cpu())/infnorm(x))    

    optimiser.zero_grad()
    y_pred = model(x_adv)
    loss = loss_fn(y_pred, y)
    loss.backward()
    optimiser.step()

    return loss, y_pred

## l2 ball

In [37]:
if TrainL2:
    ## initialize model
    adv_model_l2 = Net().to(DEVICE)
    lr = 0.1
    optimiser = optim.SGD(adv_model_l2.parameters(), lr=lr)
    epochs = 5
    ## train model
    training_history_l2 = olympic.fit(
        adv_model_l2,
        optimiser,
        nn.CrossEntropyLoss(),
        dataloader=train_loader,
        epochs=epochs,
        metrics=['accuracy'],
        prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)),
        update_fn=adversarial_training,
        update_fn_kwargs={'adversary': pgd, 'k': 10, 'step': 0.01, 'eps': 3.0, 'norm': 2, 'random':True},
        callbacks=[
            olympic.callbacks.Evaluate(val_loader),
            olympic.callbacks.ReduceLROnPlateau(patience=5, factor=0.5, min_delta=0.005, monitor='val_accuracy')
        ]
    )
    ## verify validation accuracy
    print('final validation accuracy:')
    valacc = olympic.evaluate(adv_model_l2, val_loader, metrics=['accuracy'],
                     prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)))
    print(valacc['val_accuracy'])
    ## save trained model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_AT2_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(adv_model_l2,modelname)


Epoch 1:   0%|          | 0/390 [00:00<?, ?it/s]

Begin training...



Epoch 1:  24%|██▍       | 93/390 [00:52<02:46,  1.78it/s, loss=1.84, accuracy=0.328]


Epoch 1:  42%|████▏     | 164/390 [01:33<02:19,  1.62it/s, loss=1.61, accuracy=0.398]

KeyboardInterrupt: 

## linf ball

In [24]:
if TrainLInf:
    ## initialize model
    adv_model_linf = Net().to(DEVICE)
    ## train params
    lr = 0.1
    optimiser = optim.SGD(adv_model_linf.parameters(), lr=lr)
    epochs = 5
    ## train model
    training_history_linf = olympic.fit(
        adv_model_linf,
        optimiser,
        nn.CrossEntropyLoss(),
        dataloader=train_loader,
        epochs=epochs,
        metrics=['accuracy'],
        prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)),
        update_fn=adversarial_training,
        update_fn_kwargs={'adversary': iterated_fgsm,'k': 10, 'step': 0.07, 'eps': 0.032, 'norm': 'inf', 'random':True},
        callbacks=[
            olympic.callbacks.Evaluate(val_loader),
            olympic.callbacks.ReduceLROnPlateau(patience=5, factor=0.5, min_delta=0.005, monitor='val_accuracy')
        ]
    )
    ## verify validation
    print('final validation accuracy:')
    valacc = olympic.evaluate(adv_model_linf, val_loader, metrics=['accuracy'],
                     prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)))
    
    ## save trained model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_ATInf_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(adv_model_linf,modelname)


Epoch 1:   0%|          | 0/390 [00:00<?, ?it/s]

Begin training...



Epoch 1:  24%|██▍       | 93/390 [00:51<02:45,  1.80it/s, loss=2.27, accuracy=0.156]


Epoch 1:  47%|████▋     | 185/390 [01:45<02:05,  1.64it/s, loss=2, accuracy=0.281]   


Epoch 1:  71%|███████   | 277/390 [02:41<01:08,  1.65it/s, loss=1.97, accuracy=0.297]


...

Epoch 5:  49%|████▉     | 191/390 [02:01<02:01,  1.64it/s, loss=1.64, accuracy=0.383]


Epoch 5:  73%|███████▎  | 283/390 [02:57<01:04,  1.65it/s, loss=1.69, accuracy=0.367]


Epoch 5:  96%|█████████▌| 375/390 [03:53<00:09,  1.65it/s, loss=1.86, accuracy=0.328]


Epoch 5: 100%|██████████| 390/390 [04:04<00:00,  1.60it/s, loss=1.71, accuracy=0.352, val_loss=1.31, val_accuracy=0.551]

Finished.
final validation accuracy:


# TRAIN MODEL USING SAT

In [61]:
def adversarial_training_entropy(model, optimiser, loss_fn, x, y, epoch, adversary, k, step, eps, norm, gamma):
    """Performs a single update against a specified adversary"""
    model.train()
    
    # Adversial perturbation
    #alpha = 0.8
    N = 1
    loss = 0
    for l in range(N):
        x_adv = adversary(model, x, y, loss_fn, k=k, step=step, eps=eps, norm=norm, random=True, gamma=gamma)
        
        optimiser.zero_grad()
        y_pred = model(x_adv)
        loss = loss + loss_fn(y_pred,y)
        #loss = (1-alpha)*loss + alpha*loss_fn(y_pred, y)
    loss = loss/N
    loss.backward()
    optimiser.step()
    
    return loss, y_pred

In [27]:
if TrainSAT2:
    ## initialize model
    #model_SAT2 = model_cnn().to(DEVICE)
    model_SAT2 = Net().to(DEVICE)
    ## train params
    lr = 0.1
    optimiser = optim.SGD(model_SAT2.parameters(), lr=lr)
    epochs = 5
    ## train model
    training_history_entropySmoothing = olympic.fit(
        model_SAT2,
        optimiser,
        nn.CrossEntropyLoss(),
        dataloader=train_loader,
        epochs=epochs,
        metrics=['accuracy'],
        prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)),
        update_fn=adversarial_training_entropy,
        update_fn_kwargs={'adversary': entropySmoothing, 'k': 10, 'step': 0.01, 'eps': 3.0, 'norm': 2, 'gamma':1e-5},
        callbacks=[
            olympic.callbacks.Evaluate(val_loader),
            olympic.callbacks.ReduceLROnPlateau(patience=5, factor=0.5, min_delta=0.005, monitor='val_accuracy')
        ]
    )
    ## verify validation
    print('final validation accuracy:')
    olympic.evaluate(model_SAT2, val_loader, metrics=['accuracy'],
                     prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)))
    ## save model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_SAT2_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_SAT2,modelname)


Epoch 1:   0%|          | 0/390 [00:00<?, ?it/s]

Begin training...



Epoch 1:  24%|██▍       | 93/390 [00:53<02:48,  1.76it/s, loss=1.91, accuracy=0.312]


Epoch 1:  47%|████▋     | 185/390 [01:46<02:06,  1.62it/s, loss=1.49, accuracy=0.492]


Epoch 1:  71%|███████   | 277/390 [02:43<01:06,  1.70it/s, loss=1.49, accuracy=0.391]


...

Epoch 5:  46%|████▋     | 181/390 [01:52<02:05,  1.66it/s, loss=0.754, accuracy=0.742]


Epoch 5:  70%|██████▉   | 272/390 [02:48<01:13,  1.60it/s, loss=0.665, accuracy=0.797]


Epoch 5:  93%|█████████▎| 363/390 [03:44<00:16,  1.62it/s, loss=0.627, accuracy=0.766]


Epoch 5: 100%|██████████| 390/390 [04:03<00:00,  1.60it/s, loss=0.656, accuracy=0.761, val_loss=0.781, val_accuracy=0.737]

Finished.
final validation accuracy:


In [ ]:
if TrainSATInf:
    ## initialize model
    model_SATInf = Net().to(DEVICE)
    ## train params
    #lr = 0.1
    lr = 0.05
    optimiser = optim.SGD(model_SATInf.parameters(), lr=lr)
    '''
    best model
    lr = 0.1
    epochs = 20
    k_satinf = 20
    step_satinf = 0.01
    eps_satinf = 0.2
    '''
    lr = 0.05
    epochs = 20
    k_satinf = 20
    step_satinf = 0.01
    eps_satinf = 0.2
    ## train model
    training_history_entropySmoothing = olympic.fit(
        model_SATInf,
        optimiser,
        nn.CrossEntropyLoss(),
        dataloader=train_loader,
        epochs=epochs,
        metrics=['accuracy'],
        prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)),
        update_fn=adversarial_training_entropy,
        #update_fn_kwargs={'adversary': entropySmoothing, 'k': 10, 'step': 0.01, 'eps': 0.3, 'norm': 'inf', 'gamma':1e-5},
        #update_fn_kwargs={'adversary': entropySmoothing, 'k': 20, 'step': 0.015, 'eps': 0.3, 'norm': 'inf', 'gamma':1e-5},
        #update_fn_kwargs={'adversary': entropySmoothing, 'k': 20, 'step': 0.015, 'eps': 0.3, 'norm': 'inf', 'gamma':1e-5},
        update_fn_kwargs={'adversary': entropySmoothing, 'k': k_satinf, 'step': step_satinf, 'eps': eps_satinf, 'norm': 'inf', 'gamma':1e-5},
        callbacks=[
            olympic.callbacks.Evaluate(val_loader),
            olympic.callbacks.ReduceLROnPlateau(patience=5, factor=0.5, min_delta=0.005, monitor='val_accuracy')
        ]
    )
    ## verify validation
    print('final validation accuracy:')
    olympic.evaluate(model_SATInf, val_loader, metrics=['accuracy'],
                     prepare_batch = lambda batch: (batch[0].to(DEVICE), batch[1].to(DEVICE)))
    ## save model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_SATInf_ep'+str(epochs)+'_k_'+str(k_satinf)+'_step_'+str(step_satinf)+'_lr'+str(lr)+'.pt'
    torch.save(model_SATInf,modelname)



Epoch 1:   0%|          | 0/390 [00:00<?, ?it/s]

Begin training...




Epoch 1:   0%|          | 1/390 [00:01<08:05,  1.25s/it]

Epoch 1:   0%|          | 1/390 [00:01<08:05,  1.25s/it, loss=2.61, accuracy=0]

Epoch 1:   1%|          | 2/390 [00:01<06:44,  1.04s/it, loss=2.61, accuracy=0]

Epoch 1:   1%|          | 2/390 [00:01<06:44,  1.04s/it, loss=2.74, accuracy=0.0938]

Epoch 1:   1%|          | 3/390 [00:02<05:47,  1.11it/s, loss=2.74, accuracy=0.0938]

Epoch 1:   1%|          | 3/390 [00:02<05:47,  1.11it/s, loss=2.79, accuracy=0.0938]

Epoch 1:   1%|          | 4/390 [00:02<05:07,  1.25it/s, loss=2.79, accuracy=0.0938]

Epoch 1:   1%|          | 4/390 [00:02<05:07,  1.25it/s, loss=2.62, accuracy=0.109] 

Epoch 1:   1%|▏         | 5/390 [00:03<04:40,  1.37it/s, loss=2.62, accuracy=0.109]

Epoch 1:   1%|▏         | 5/390 [00:03<04:40,  1.37it/s, loss=2.56, accuracy=0.172]

Epoch 1:   2%|▏         | 6/390 [00:04<04:20,  1.48it/s, loss=2.56, accuracy=0.172]

Epoch 1:   2%|▏         | 6/390 [00:04<04:20,  1.48it/s, loss=2.26, accuracy=0.0703]

Epoch 1

Epoch 1:  23%|██▎       | 90/390 [00:51<02:51,  1.75it/s, loss=1.83, accuracy=0.266]

Epoch 1:  23%|██▎       | 90/390 [00:51<02:51,  1.75it/s, loss=1.83, accuracy=0.32] 

Epoch 1:  23%|██▎       | 91/390 [00:52<02:51,  1.75it/s, loss=1.83, accuracy=0.32]

Epoch 1:  23%|██▎       | 91/390 [00:52<02:51,  1.75it/s, loss=1.73, accuracy=0.375]

Epoch 1:  24%|██▎       | 92/390 [00:52<02:50,  1.74it/s, loss=1.73, accuracy=0.375]

Epoch 1:  24%|██▎       | 92/390 [00:52<02:50,  1.74it/s, loss=1.59, accuracy=0.367]

Epoch 1:  24%|██▍       | 93/390 [00:53<02:52,  1.73it/s, loss=1.59, accuracy=0.367]

Epoch 1:  24%|██▍       | 93/390 [00:53<02:52,  1.73it/s, loss=1.91, accuracy=0.328]

Epoch 1:  24%|██▍       | 94/390 [00:54<02:52,  1.72it/s, loss=1.91, accuracy=0.328]

Epoch 1:  24%|██▍       | 94/390 [00:54<02:52,  1.72it/s, loss=1.75, accuracy=0.344]

Epoch 1:  24%|██▍       | 95/390 [00:54<02:51,  1.72it/s, loss=1.75, accuracy=0.344]

Epoch 1:  24%|██▍       | 95/390 [00:54<02:51,  1.72it/

Epoch 1:  46%|████▌     | 178/390 [01:46<02:12,  1.60it/s, loss=1.66, accuracy=0.398]

Epoch 1:  46%|████▌     | 178/390 [01:46<02:12,  1.60it/s, loss=1.75, accuracy=0.312]

Epoch 1:  46%|████▌     | 179/390 [01:46<02:12,  1.60it/s, loss=1.75, accuracy=0.312]

Epoch 1:  46%|████▌     | 179/390 [01:46<02:12,  1.60it/s, loss=1.76, accuracy=0.328]

Epoch 1:  46%|████▌     | 180/390 [01:47<02:11,  1.60it/s, loss=1.76, accuracy=0.328]

Epoch 1:  46%|████▌     | 180/390 [01:47<02:11,  1.60it/s, loss=1.71, accuracy=0.383]

Epoch 1:  46%|████▋     | 181/390 [01:48<02:10,  1.60it/s, loss=1.71, accuracy=0.383]

Epoch 1:  46%|████▋     | 181/390 [01:48<02:10,  1.60it/s, loss=1.6, accuracy=0.359] 

Epoch 1:  47%|████▋     | 182/390 [01:48<02:10,  1.60it/s, loss=1.6, accuracy=0.359]

Epoch 1:  47%|████▋     | 182/390 [01:48<02:10,  1.60it/s, loss=1.63, accuracy=0.391]

Epoch 1:  47%|████▋     | 183/390 [01:49<02:09,  1.59it/s, loss=1.63, accuracy=0.391]

Epoch 1:  47%|████▋     | 183/390 [01:49<02:

...

Epoch 3:   4%|▎         | 14/390 [00:09<03:56,  1.59it/s, loss=1.39, accuracy=0.562]

Epoch 3:   4%|▎         | 14/390 [00:09<03:56,  1.59it/s, loss=1.47, accuracy=0.5]  

Epoch 3:   4%|▍         | 15/390 [00:09<03:55,  1.59it/s, loss=1.47, accuracy=0.5]

Epoch 3:   4%|▍         | 15/390 [00:09<03:55,  1.59it/s, loss=1.17, accuracy=0.617]

Epoch 3:   4%|▍         | 16/390 [00:10<03:53,  1.60it/s, loss=1.17, accuracy=0.617]

Epoch 3:   4%|▍         | 16/390 [00:10<03:53,  1.60it/s, loss=1.45, accuracy=0.43] 

Epoch 3:   4%|▍         | 17/390 [00:11<03:50,  1.62it/s, loss=1.45, accuracy=0.43]

Epoch 3:   4%|▍         | 17/390 [00:11<03:50,  1.62it/s, loss=1.16, accuracy=0.562]

Epoch 3:   5%|▍         | 18/390 [00:11<03:47,  1.64it/s, loss=1.16, accuracy=0.562]

Epoch 3:   5%|▍         | 18/390 [00:11<03:47,  1.64it/s, loss=1.23, accuracy=0.562]

Epoch 3:   5%|▍         | 19/390 [00:12<03:44,  1.65it/s, loss=1.23, accuracy=0.562]

Epoch 3:   5%|▍         | 19/390 [00:12<03:44,  1.65it/s,

Epoch 3:  26%|██▋       | 103/390 [01:04<02:57,  1.62it/s, loss=1.31, accuracy=0.57]

Epoch 3:  26%|██▋       | 103/390 [01:04<02:57,  1.62it/s, loss=1.3, accuracy=0.516]

Epoch 3:  27%|██▋       | 104/390 [01:04<02:55,  1.63it/s, loss=1.3, accuracy=0.516]

Epoch 3:  27%|██▋       | 104/390 [01:04<02:55,  1.63it/s, loss=1.32, accuracy=0.523]

Epoch 3:  27%|██▋       | 105/390 [01:05<02:53,  1.64it/s, loss=1.32, accuracy=0.523]

Epoch 3:  27%|██▋       | 105/390 [01:05<02:53,  1.64it/s, loss=1.35, accuracy=0.5]  

Epoch 3:  27%|██▋       | 106/390 [01:06<02:52,  1.64it/s, loss=1.35, accuracy=0.5]

Epoch 3:  27%|██▋       | 106/390 [01:06<02:52,  1.64it/s, loss=1.13, accuracy=0.57]

Epoch 3:  27%|██▋       | 107/390 [01:06<02:51,  1.65it/s, loss=1.13, accuracy=0.57]

Epoch 3:  27%|██▋       | 107/390 [01:06<02:51,  1.65it/s, loss=1.34, accuracy=0.539]

Epoch 3:  28%|██▊       | 108/390 [01:07<02:50,  1.65it/s, loss=1.34, accuracy=0.539]

Epoch 3:  28%|██▊       | 108/390 [01:07<02:50,  1

Epoch 3:  49%|████▉     | 191/390 [01:58<02:04,  1.60it/s, loss=1.31, accuracy=0.516]

Epoch 3:  49%|████▉     | 191/390 [01:58<02:04,  1.60it/s, loss=1.2, accuracy=0.531] 

Epoch 3:  49%|████▉     | 192/390 [01:59<02:04,  1.60it/s, loss=1.2, accuracy=0.531]

Epoch 3:  49%|████▉     | 192/390 [01:59<02:04,  1.60it/s, loss=1.13, accuracy=0.586]

Epoch 3:  49%|████▉     | 193/390 [02:00<02:03,  1.60it/s, loss=1.13, accuracy=0.586]

Epoch 3:  49%|████▉     | 193/390 [02:00<02:03,  1.60it/s, loss=1.09, accuracy=0.625]

Epoch 3:  50%|████▉     | 194/390 [02:00<02:01,  1.61it/s, loss=1.09, accuracy=0.625]

Epoch 3:  50%|████▉     | 194/390 [02:00<02:01,  1.61it/s, loss=1.28, accuracy=0.594]

Epoch 3:  50%|█████     | 195/390 [02:01<02:00,  1.61it/s, loss=1.28, accuracy=0.594]

Epoch 3:  50%|█████     | 195/390 [02:01<02:00,  1.61it/s, loss=1.34, accuracy=0.477]

Epoch 3:  50%|█████     | 196/390 [02:02<02:00,  1.61it/s, loss=1.34, accuracy=0.477]

Epoch 3:  50%|█████     | 196/390 [02:02<02:

Epoch 3:  72%|███████▏  | 279/390 [02:53<01:09,  1.60it/s, loss=1.15, accuracy=0.562]

Epoch 3:  72%|███████▏  | 279/390 [02:53<01:09,  1.60it/s, loss=1.25, accuracy=0.547]

Epoch 3:  72%|███████▏  | 280/390 [02:54<01:08,  1.61it/s, loss=1.25, accuracy=0.547]

Epoch 3:  72%|███████▏  | 280/390 [02:54<01:08,  1.61it/s, loss=1.2, accuracy=0.617] 

Epoch 3:  72%|███████▏  | 281/390 [02:54<01:08,  1.60it/s, loss=1.2, accuracy=0.617]

Epoch 3:  72%|███████▏  | 281/390 [02:54<01:08,  1.60it/s, loss=1.11, accuracy=0.555]

Epoch 3:  72%|███████▏  | 282/390 [02:55<01:07,  1.60it/s, loss=1.11, accuracy=0.555]

Epoch 3:  72%|███████▏  | 282/390 [02:55<01:07,  1.60it/s, loss=1.19, accuracy=0.594]

Epoch 3:  73%|███████▎  | 283/390 [02:55<01:06,  1.60it/s, loss=1.19, accuracy=0.594]

Epoch 3:  73%|███████▎  | 283/390 [02:55<01:06,  1.60it/s, loss=1.25, accuracy=0.555]

Epoch 3:  73%|███████▎  | 284/390 [02:56<01:06,  1.60it/s, loss=1.25, accuracy=0.555]

Epoch 3:  73%|███████▎  | 284/390 [02:56<01:

Epoch 3:  94%|█████████▍| 367/390 [03:48<00:14,  1.60it/s, loss=1.02, accuracy=0.609]

Epoch 3:  94%|█████████▍| 367/390 [03:48<00:14,  1.60it/s, loss=1.23, accuracy=0.594]

Epoch 3:  94%|█████████▍| 368/390 [03:48<00:13,  1.59it/s, loss=1.23, accuracy=0.594]

Epoch 3:  94%|█████████▍| 368/390 [03:48<00:13,  1.59it/s, loss=1.3, accuracy=0.57]  

Epoch 3:  95%|█████████▍| 369/390 [03:49<00:13,  1.59it/s, loss=1.3, accuracy=0.57]

Epoch 3:  95%|█████████▍| 369/390 [03:49<00:13,  1.59it/s, loss=1.02, accuracy=0.625]

Epoch 3:  95%|█████████▍| 370/390 [03:50<00:12,  1.59it/s, loss=1.02, accuracy=0.625]

Epoch 3:  95%|█████████▍| 370/390 [03:50<00:12,  1.59it/s, loss=1.06, accuracy=0.625]

Epoch 3:  95%|█████████▌| 371/390 [03:50<00:11,  1.59it/s, loss=1.06, accuracy=0.625]

Epoch 3:  95%|█████████▌| 371/390 [03:50<00:11,  1.59it/s, loss=1.08, accuracy=0.625]

Epoch 3:  95%|█████████▌| 372/390 [03:51<00:11,  1.59it/s, loss=1.08, accuracy=0.625]

Epoch 3:  95%|█████████▌| 372/390 [03:51<00:1

# TRAIN MODEL USING TRADES

In [30]:
args = {}
args['test_batch_size'] = 128
args['no_cuda'] = False
args['epsilon'] = 0.3
args['num_steps'] = 5
args['step_size'] = 0.01
args['random'] =True,
args['white_box_attack']=True
args['log_interval'] = 100
args['beta'] = 1.0
args['log_interval'] = 1

In [31]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()

        # calculate robust loss
        loss = trades_loss(model=model,
                           x_natural=data,
                           y=target,
                           optimizer=optimizer,
                           step_size=args['step_size'],
                           epsilon=args['epsilon'],
                           perturb_steps=args['num_steps'],
                           beta=args['beta'],
                           distance = 'l_2')
        

        #print('outloss pre step:',loss)
        loss.backward(retain_graph=True)
        
        optimizer.step()
        #print('outloss post step:',loss.item())

        # print progress
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [32]:
def eval_train(model, device, train_loader):
    model.eval()
    train_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            train_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    train_loss /= len(train_loader.dataset)
    print('Training: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    training_accuracy = correct / len(train_loader.dataset)
    return train_loss, training_accuracy

In [33]:
def eval_test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_accuracy = correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [34]:
if TrainTRADES:
    args['attack'] = 'l_2' #or 'l_inf'
    ## initialize model
    #model_TRADES = model_cnn().to(DEVICE)
    model_TRADES = Net().to(DEVICE)
    ## training params
    lr = 0.1
    optimizer = optim.SGD(model_TRADES.parameters(), lr=lr)
    epochs = 10
    ## train model

    for epoch in range(1, epochs + 1):
        # adjust learning rate for SGD
        #adjust_learning_rate(optimizer, epoch)

        # adversarial training
        train(args, model_TRADES, DEVICE, train_loader, optimizer, epoch)

        # evaluation on natural examples
        print('================================================================')
        eval_train(model_TRADES, DEVICE, train_loader)
        eval_test(model_TRADES, DEVICE, val_loader)
        print('================================================================')

    ## save model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_TRADES_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_TRADES,modelname)

/home/abc586/anaconda3/envs/deepsoft/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.485212
Train Epoch: 1 [128/50000 (0%)]	Loss: 3.861271
Train Epoch: 1 [256/50000 (1%)]	Loss: 4.884131
Train Epoch: 1 [384/50000 (1%)]	Loss: 3.484039
Train Epoch: 1 [512/50000 (1%)]	Loss: 2.673924
Train Epoch: 1 [640/50000 (1%)]	Loss: 2.770136
Train Epoch: 1 [768/50000 (2%)]	Loss: 3.520533
Train Epoch: 1 [896/50000 (2%)]	Loss: 2.659272
Train Epoch: 1 [1024/50000 (2%)]	Loss: 2.300247
Train Epoch: 1 [1152/50000 (2%)]	Loss: 2.234285
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.075466
Train Epoch: 1 [1408/50000 (3%)]	Loss: 2.241072
Train Epoch: 1 [1536/50000 (3%)]	Loss: 2.069472
Train Epoch: 1 [1664/50000 (3%)]	Loss: 2.125877
Train Epoch: 1 [1792/50000 (4%)]	Loss: 2.106936
Train Epoch: 1 [1920/50000 (4%)]	Loss: 2.164867
Train Epoch: 1 [2048/50000 (4%)]	Loss: 1.851210
Train Epoch: 1 [2176/50000 (4%)]	Loss: 2.060441
Train Epoch: 1 [2304/50000 (5%)]	Loss: 2.047996
Train Epoch: 1 [2432/50000 (5%)]	Loss: 2.229149
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.026534
Tr

Train Epoch: 1 [21376/50000 (43%)]	Loss: 1.573278
Train Epoch: 1 [21504/50000 (43%)]	Loss: 1.443508
Train Epoch: 1 [21632/50000 (43%)]	Loss: 1.394219
Train Epoch: 1 [21760/50000 (44%)]	Loss: 1.403605
Train Epoch: 1 [21888/50000 (44%)]	Loss: 1.376626
Train Epoch: 1 [22016/50000 (44%)]	Loss: 1.371379
Train Epoch: 1 [22144/50000 (44%)]	Loss: 1.450980
Train Epoch: 1 [22272/50000 (45%)]	Loss: 1.721959
Train Epoch: 1 [22400/50000 (45%)]	Loss: 1.704408
Train Epoch: 1 [22528/50000 (45%)]	Loss: 1.372382
Train Epoch: 1 [22656/50000 (45%)]	Loss: 1.513580
Train Epoch: 1 [22784/50000 (46%)]	Loss: 1.476556
Train Epoch: 1 [22912/50000 (46%)]	Loss: 1.393982
Train Epoch: 1 [23040/50000 (46%)]	Loss: 1.323953
Train Epoch: 1 [23168/50000 (46%)]	Loss: 1.538219
Train Epoch: 1 [23296/50000 (47%)]	Loss: 1.227866
Train Epoch: 1 [23424/50000 (47%)]	Loss: 1.536891
Train Epoch: 1 [23552/50000 (47%)]	Loss: 1.355719
Train Epoch: 1 [23680/50000 (47%)]	Loss: 1.288554
Train Epoch: 1 [23808/50000 (48%)]	Loss: 1.402622


Train Epoch: 1 [42368/50000 (85%)]	Loss: 1.092708
Train Epoch: 1 [42496/50000 (85%)]	Loss: 1.563728
Train Epoch: 1 [42624/50000 (85%)]	Loss: 1.521305
Train Epoch: 1 [42752/50000 (86%)]	Loss: 1.372478
Train Epoch: 1 [42880/50000 (86%)]	Loss: 1.282337
Train Epoch: 1 [43008/50000 (86%)]	Loss: 1.068817
Train Epoch: 1 [43136/50000 (86%)]	Loss: 1.341169
Train Epoch: 1 [43264/50000 (87%)]	Loss: 1.241365
Train Epoch: 1 [43392/50000 (87%)]	Loss: 1.257902
Train Epoch: 1 [43520/50000 (87%)]	Loss: 1.266996
Train Epoch: 1 [43648/50000 (87%)]	Loss: 1.337126
Train Epoch: 1 [43776/50000 (88%)]	Loss: 1.285730
Train Epoch: 1 [43904/50000 (88%)]	Loss: 1.149737
Train Epoch: 1 [44032/50000 (88%)]	Loss: 1.366921
Train Epoch: 1 [44160/50000 (88%)]	Loss: 1.452355
Train Epoch: 1 [44288/50000 (89%)]	Loss: 1.369191
Train Epoch: 1 [44416/50000 (89%)]	Loss: 1.333575
Train Epoch: 1 [44544/50000 (89%)]	Loss: 1.137673
Train Epoch: 1 [44672/50000 (89%)]	Loss: 1.245307
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.199284


Train Epoch: 2 [13184/50000 (26%)]	Loss: 1.148302
Train Epoch: 2 [13312/50000 (27%)]	Loss: 1.044853
Train Epoch: 2 [13440/50000 (27%)]	Loss: 0.963983
Train Epoch: 2 [13568/50000 (27%)]	Loss: 1.113124
Train Epoch: 2 [13696/50000 (27%)]	Loss: 1.296980
Train Epoch: 2 [13824/50000 (28%)]	Loss: 1.176069
Train Epoch: 2 [13952/50000 (28%)]	Loss: 1.019639
Train Epoch: 2 [14080/50000 (28%)]	Loss: 1.180943
Train Epoch: 2 [14208/50000 (28%)]	Loss: 1.073123
Train Epoch: 2 [14336/50000 (29%)]	Loss: 1.073352
Train Epoch: 2 [14464/50000 (29%)]	Loss: 1.026852
Train Epoch: 2 [14592/50000 (29%)]	Loss: 1.195084
Train Epoch: 2 [14720/50000 (29%)]	Loss: 1.051529
Train Epoch: 2 [14848/50000 (30%)]	Loss: 1.117998
Train Epoch: 2 [14976/50000 (30%)]	Loss: 1.144975
Train Epoch: 2 [15104/50000 (30%)]	Loss: 1.313738
Train Epoch: 2 [15232/50000 (31%)]	Loss: 1.130858
Train Epoch: 2 [15360/50000 (31%)]	Loss: 0.971401
Train Epoch: 2 [15488/50000 (31%)]	Loss: 1.178351
Train Epoch: 2 [15616/50000 (31%)]	Loss: 1.118664


...

Train Epoch: 8 [48000/50000 (96%)]	Loss: 0.160368
Train Epoch: 8 [48128/50000 (96%)]	Loss: 0.147811
Train Epoch: 8 [48256/50000 (97%)]	Loss: 0.216222
Train Epoch: 8 [48384/50000 (97%)]	Loss: 0.102173
Train Epoch: 8 [48512/50000 (97%)]	Loss: 0.094139
Train Epoch: 8 [48640/50000 (97%)]	Loss: 0.135851
Train Epoch: 8 [48768/50000 (98%)]	Loss: 0.126412
Train Epoch: 8 [48896/50000 (98%)]	Loss: 0.143412
Train Epoch: 8 [49024/50000 (98%)]	Loss: 0.158834
Train Epoch: 8 [49152/50000 (98%)]	Loss: 0.109769
Train Epoch: 8 [49280/50000 (99%)]	Loss: 0.152412
Train Epoch: 8 [49408/50000 (99%)]	Loss: 0.155604
Train Epoch: 8 [49536/50000 (99%)]	Loss: 0.150239
Train Epoch: 8 [49664/50000 (99%)]	Loss: 0.179992
Train Epoch: 8 [49792/50000 (100%)]	Loss: 0.164443
Training: Average loss: 0.2510, Accuracy: 45631/50000 (91%)
Test: Average loss: 1.0003, Accuracy: 7333/10000 (73%)
Train Epoch: 9 [0/50000 (0%)]	Loss: 0.167950
Train Epoch: 9 [128/50000 (0%)]	Loss: 0.210616
Train Epoch: 9 [256/50000 (1%)]	Loss: 0.14

Train Epoch: 9 [18816/50000 (38%)]	Loss: 0.152259
Train Epoch: 9 [18944/50000 (38%)]	Loss: 0.185349
Train Epoch: 9 [19072/50000 (38%)]	Loss: 0.138526
Train Epoch: 9 [19200/50000 (38%)]	Loss: 0.160312
Train Epoch: 9 [19328/50000 (39%)]	Loss: 0.130466
Train Epoch: 9 [19456/50000 (39%)]	Loss: 0.135391
Train Epoch: 9 [19584/50000 (39%)]	Loss: 0.161445
Train Epoch: 9 [19712/50000 (39%)]	Loss: 0.152983
Train Epoch: 9 [19840/50000 (40%)]	Loss: 0.185906
Train Epoch: 9 [19968/50000 (40%)]	Loss: 0.201903
Train Epoch: 9 [20096/50000 (40%)]	Loss: 0.146513
Train Epoch: 9 [20224/50000 (41%)]	Loss: 0.290082
Train Epoch: 9 [20352/50000 (41%)]	Loss: 0.290126
Train Epoch: 9 [20480/50000 (41%)]	Loss: 0.157073
Train Epoch: 9 [20608/50000 (41%)]	Loss: 0.191855
Train Epoch: 9 [20736/50000 (42%)]	Loss: 0.185486
Train Epoch: 9 [20864/50000 (42%)]	Loss: 0.124039
Train Epoch: 9 [20992/50000 (42%)]	Loss: 0.178239
Train Epoch: 9 [21120/50000 (42%)]	Loss: 0.210613
Train Epoch: 9 [21248/50000 (43%)]	Loss: 0.189576


Train Epoch: 9 [39808/50000 (80%)]	Loss: 0.093999
Train Epoch: 9 [39936/50000 (80%)]	Loss: 0.133309
Train Epoch: 9 [40064/50000 (80%)]	Loss: 0.168691
Train Epoch: 9 [40192/50000 (81%)]	Loss: 0.110830
Train Epoch: 9 [40320/50000 (81%)]	Loss: 0.221308
Train Epoch: 9 [40448/50000 (81%)]	Loss: 0.148260
Train Epoch: 9 [40576/50000 (81%)]	Loss: 0.166342
Train Epoch: 9 [40704/50000 (82%)]	Loss: 0.169400
Train Epoch: 9 [40832/50000 (82%)]	Loss: 0.197433
Train Epoch: 9 [40960/50000 (82%)]	Loss: 0.145630
Train Epoch: 9 [41088/50000 (82%)]	Loss: 0.226676
Train Epoch: 9 [41216/50000 (83%)]	Loss: 0.226382
Train Epoch: 9 [41344/50000 (83%)]	Loss: 0.229074
Train Epoch: 9 [41472/50000 (83%)]	Loss: 0.221701
Train Epoch: 9 [41600/50000 (83%)]	Loss: 0.154221
Train Epoch: 9 [41728/50000 (84%)]	Loss: 0.192796
Train Epoch: 9 [41856/50000 (84%)]	Loss: 0.150225
Train Epoch: 9 [41984/50000 (84%)]	Loss: 0.150576
Train Epoch: 9 [42112/50000 (84%)]	Loss: 0.138230
Train Epoch: 9 [42240/50000 (85%)]	Loss: 0.172541


Train Epoch: 10 [10368/50000 (21%)]	Loss: 0.126457
Train Epoch: 10 [10496/50000 (21%)]	Loss: 0.094488
Train Epoch: 10 [10624/50000 (21%)]	Loss: 0.122734
Train Epoch: 10 [10752/50000 (22%)]	Loss: 0.092059
Train Epoch: 10 [10880/50000 (22%)]	Loss: 0.116114
Train Epoch: 10 [11008/50000 (22%)]	Loss: 0.153121
Train Epoch: 10 [11136/50000 (22%)]	Loss: 0.162793
Train Epoch: 10 [11264/50000 (23%)]	Loss: 0.127693
Train Epoch: 10 [11392/50000 (23%)]	Loss: 0.094768
Train Epoch: 10 [11520/50000 (23%)]	Loss: 0.100085
Train Epoch: 10 [11648/50000 (23%)]	Loss: 0.104437
Train Epoch: 10 [11776/50000 (24%)]	Loss: 0.111882
Train Epoch: 10 [11904/50000 (24%)]	Loss: 0.067344
Train Epoch: 10 [12032/50000 (24%)]	Loss: 0.132846
Train Epoch: 10 [12160/50000 (24%)]	Loss: 0.117053
Train Epoch: 10 [12288/50000 (25%)]	Loss: 0.144722
Train Epoch: 10 [12416/50000 (25%)]	Loss: 0.097094
Train Epoch: 10 [12544/50000 (25%)]	Loss: 0.108559
Train Epoch: 10 [12672/50000 (25%)]	Loss: 0.143756
Train Epoch: 10 [12800/50000 (2

Train Epoch: 10 [30976/50000 (62%)]	Loss: 0.150088
Train Epoch: 10 [31104/50000 (62%)]	Loss: 0.151596
Train Epoch: 10 [31232/50000 (63%)]	Loss: 0.070708
Train Epoch: 10 [31360/50000 (63%)]	Loss: 0.211655
Train Epoch: 10 [31488/50000 (63%)]	Loss: 0.115551
Train Epoch: 10 [31616/50000 (63%)]	Loss: 0.138033
Train Epoch: 10 [31744/50000 (64%)]	Loss: 0.122092
Train Epoch: 10 [31872/50000 (64%)]	Loss: 0.098329
Train Epoch: 10 [32000/50000 (64%)]	Loss: 0.099995
Train Epoch: 10 [32128/50000 (64%)]	Loss: 0.119207
Train Epoch: 10 [32256/50000 (65%)]	Loss: 0.106969
Train Epoch: 10 [32384/50000 (65%)]	Loss: 0.136093
Train Epoch: 10 [32512/50000 (65%)]	Loss: 0.119558
Train Epoch: 10 [32640/50000 (65%)]	Loss: 0.098115
Train Epoch: 10 [32768/50000 (66%)]	Loss: 0.146366
Train Epoch: 10 [32896/50000 (66%)]	Loss: 0.109537
Train Epoch: 10 [33024/50000 (66%)]	Loss: 0.132640
Train Epoch: 10 [33152/50000 (66%)]	Loss: 0.122114
Train Epoch: 10 [33280/50000 (67%)]	Loss: 0.118010
Train Epoch: 10 [33408/50000 (6

In [38]:
if TrainTRADESInf:
    args['attack'] = 'l_inf' 
    ## initialize model
    model_TRADESInf = Net().to(DEVICE)
    ## training params
    lr = 0.1
    optimizer = optim.SGD(model_TRADESInf.parameters(), lr=lr)
    epochs = 10
    ## train model

    for epoch in range(1, epochs + 1):
        # adjust learning rate for SGD
        #adjust_learning_rate(optimizer, epoch)

        # adversarial training
        train(args, model_TRADESInf, DEVICE, train_loader, optimizer, epoch)

        # evaluation on natural examples
        print('================================================================')
        eval_train(model_TRADESInf, DEVICE, train_loader)
        eval_test(model_TRADESInf, DEVICE, val_loader)
        print('================================================================')

    ## save model
    modelname = '../trainedmodels/'+dataset+'/'+NetName+'_TRADESInf_ep'+str(epochs)+'_lr'+str(lr)+'.pt'
    torch.save(model_TRADESInf,modelname)

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.423514
Train Epoch: 1 [128/50000 (0%)]	Loss: 3.420725
Train Epoch: 1 [256/50000 (1%)]	Loss: 5.006063
Train Epoch: 1 [384/50000 (1%)]	Loss: 3.384504
Train Epoch: 1 [512/50000 (1%)]	Loss: 3.307767
Train Epoch: 1 [640/50000 (1%)]	Loss: 2.890934
Train Epoch: 1 [768/50000 (2%)]	Loss: 3.227223
Train Epoch: 1 [896/50000 (2%)]	Loss: 2.973078
Train Epoch: 1 [1024/50000 (2%)]	Loss: 2.536717
Train Epoch: 1 [1152/50000 (2%)]	Loss: 2.143846
Train Epoch: 1 [1280/50000 (3%)]	Loss: 2.030766
Train Epoch: 1 [1408/50000 (3%)]	Loss: 2.177700
Train Epoch: 1 [1536/50000 (3%)]	Loss: 1.995935
Train Epoch: 1 [1664/50000 (3%)]	Loss: 2.118494
Train Epoch: 1 [1792/50000 (4%)]	Loss: 2.087103
Train Epoch: 1 [1920/50000 (4%)]	Loss: 2.113254
Train Epoch: 1 [2048/50000 (4%)]	Loss: 1.885515
Train Epoch: 1 [2176/50000 (4%)]	Loss: 2.155241
Train Epoch: 1 [2304/50000 (5%)]	Loss: 2.100457
Train Epoch: 1 [2432/50000 (5%)]	Loss: 2.116394
Train Epoch: 1 [2560/50000 (5%)]	Loss: 2.053553
Tr

Train Epoch: 1 [21376/50000 (43%)]	Loss: 1.577940
Train Epoch: 1 [21504/50000 (43%)]	Loss: 1.399937
Train Epoch: 1 [21632/50000 (43%)]	Loss: 1.308321
Train Epoch: 1 [21760/50000 (44%)]	Loss: 1.390254
Train Epoch: 1 [21888/50000 (44%)]	Loss: 1.315886
Train Epoch: 1 [22016/50000 (44%)]	Loss: 1.326812
Train Epoch: 1 [22144/50000 (44%)]	Loss: 1.433020
Train Epoch: 1 [22272/50000 (45%)]	Loss: 1.822091
Train Epoch: 1 [22400/50000 (45%)]	Loss: 1.736052
Train Epoch: 1 [22528/50000 (45%)]	Loss: 1.410587
Train Epoch: 1 [22656/50000 (45%)]	Loss: 1.479423
Train Epoch: 1 [22784/50000 (46%)]	Loss: 1.427108
Train Epoch: 1 [22912/50000 (46%)]	Loss: 1.328133
Train Epoch: 1 [23040/50000 (46%)]	Loss: 1.228755
Train Epoch: 1 [23168/50000 (46%)]	Loss: 1.454886
Train Epoch: 1 [23296/50000 (47%)]	Loss: 1.198228
Train Epoch: 1 [23424/50000 (47%)]	Loss: 1.556590
Train Epoch: 1 [23552/50000 (47%)]	Loss: 1.407461
Train Epoch: 1 [23680/50000 (47%)]	Loss: 1.297973
Train Epoch: 1 [23808/50000 (48%)]	Loss: 1.444147


Train Epoch: 1 [42368/50000 (85%)]	Loss: 1.119510
Train Epoch: 1 [42496/50000 (85%)]	Loss: 1.453006
Train Epoch: 1 [42624/50000 (85%)]	Loss: 1.470349
Train Epoch: 1 [42752/50000 (86%)]	Loss: 1.271193
Train Epoch: 1 [42880/50000 (86%)]	Loss: 1.245995
Train Epoch: 1 [43008/50000 (86%)]	Loss: 1.023486
Train Epoch: 1 [43136/50000 (86%)]	Loss: 1.268650
Train Epoch: 1 [43264/50000 (87%)]	Loss: 1.198109
Train Epoch: 1 [43392/50000 (87%)]	Loss: 1.231154
Train Epoch: 1 [43520/50000 (87%)]	Loss: 1.157931
Train Epoch: 1 [43648/50000 (87%)]	Loss: 1.227131
Train Epoch: 1 [43776/50000 (88%)]	Loss: 1.326643
Train Epoch: 1 [43904/50000 (88%)]	Loss: 1.091740
Train Epoch: 1 [44032/50000 (88%)]	Loss: 1.267061
Train Epoch: 1 [44160/50000 (88%)]	Loss: 1.362897
Train Epoch: 1 [44288/50000 (89%)]	Loss: 1.387390
Train Epoch: 1 [44416/50000 (89%)]	Loss: 1.303731
Train Epoch: 1 [44544/50000 (89%)]	Loss: 1.096763
Train Epoch: 1 [44672/50000 (89%)]	Loss: 1.182177
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.186420


Train Epoch: 2 [13184/50000 (26%)]	Loss: 1.171124
Train Epoch: 2 [13312/50000 (27%)]	Loss: 1.083776
Train Epoch: 2 [13440/50000 (27%)]	Loss: 0.972932
Train Epoch: 2 [13568/50000 (27%)]	Loss: 1.054038
Train Epoch: 2 [13696/50000 (27%)]	Loss: 1.241826
Train Epoch: 2 [13824/50000 (28%)]	Loss: 1.143949
Train Epoch: 2 [13952/50000 (28%)]	Loss: 1.018252
Train Epoch: 2 [14080/50000 (28%)]	Loss: 1.166713
Train Epoch: 2 [14208/50000 (28%)]	Loss: 1.055054
Train Epoch: 2 [14336/50000 (29%)]	Loss: 0.961710
Train Epoch: 2 [14464/50000 (29%)]	Loss: 0.978195
Train Epoch: 2 [14592/50000 (29%)]	Loss: 1.141673
Train Epoch: 2 [14720/50000 (29%)]	Loss: 1.025391
Train Epoch: 2 [14848/50000 (30%)]	Loss: 1.093628
Train Epoch: 2 [14976/50000 (30%)]	Loss: 1.174139
Train Epoch: 2 [15104/50000 (30%)]	Loss: 1.230818
Train Epoch: 2 [15232/50000 (31%)]	Loss: 1.027641
Train Epoch: 2 [15360/50000 (31%)]	Loss: 0.924639
Train Epoch: 2 [15488/50000 (31%)]	Loss: 1.136788
Train Epoch: 2 [15616/50000 (31%)]	Loss: 1.125684


Train Epoch: 2 [34176/50000 (68%)]	Loss: 0.857567
Train Epoch: 2 [34304/50000 (69%)]	Loss: 1.035361
Train Epoch: 2 [34432/50000 (69%)]	Loss: 1.031340
Train Epoch: 2 [34560/50000 (69%)]	Loss: 0.966756
Train Epoch: 2 [34688/50000 (69%)]	Loss: 0.943979
Train Epoch: 2 [34816/50000 (70%)]	Loss: 0.839022
Train Epoch: 2 [34944/50000 (70%)]	Loss: 0.959636
Train Epoch: 2 [35072/50000 (70%)]	Loss: 0.864104
Train Epoch: 2 [35200/50000 (71%)]	Loss: 0.901851
Train Epoch: 2 [35328/50000 (71%)]	Loss: 0.899388
Train Epoch: 2 [35456/50000 (71%)]	Loss: 0.758961
Train Epoch: 2 [35584/50000 (71%)]	Loss: 0.956706
Train Epoch: 2 [35712/50000 (72%)]	Loss: 0.983936
Train Epoch: 2 [35840/50000 (72%)]	Loss: 0.839994
Train Epoch: 2 [35968/50000 (72%)]	Loss: 0.955749
Train Epoch: 2 [36096/50000 (72%)]	Loss: 0.902713
Train Epoch: 2 [36224/50000 (73%)]	Loss: 0.891690
Train Epoch: 2 [36352/50000 (73%)]	Loss: 0.794220
Train Epoch: 2 [36480/50000 (73%)]	Loss: 0.940860
Train Epoch: 2 [36608/50000 (73%)]	Loss: 0.942611


...

Train Epoch: 8 [48000/50000 (96%)]	Loss: 0.215974
Train Epoch: 8 [48128/50000 (96%)]	Loss: 0.214897
Train Epoch: 8 [48256/50000 (97%)]	Loss: 0.126725
Train Epoch: 8 [48384/50000 (97%)]	Loss: 0.119888
Train Epoch: 8 [48512/50000 (97%)]	Loss: 0.145639
Train Epoch: 8 [48640/50000 (97%)]	Loss: 0.168304
Train Epoch: 8 [48768/50000 (98%)]	Loss: 0.172599
Train Epoch: 8 [48896/50000 (98%)]	Loss: 0.094017
Train Epoch: 8 [49024/50000 (98%)]	Loss: 0.152665
Train Epoch: 8 [49152/50000 (98%)]	Loss: 0.169812
Train Epoch: 8 [49280/50000 (99%)]	Loss: 0.180450
Train Epoch: 8 [49408/50000 (99%)]	Loss: 0.231838
Train Epoch: 8 [49536/50000 (99%)]	Loss: 0.152455
Train Epoch: 8 [49664/50000 (99%)]	Loss: 0.130444
Train Epoch: 8 [49792/50000 (100%)]	Loss: 0.150397
Training: Average loss: 0.1552, Accuracy: 47377/50000 (95%)
Test: Average loss: 0.8265, Accuracy: 7608/10000 (76%)
Train Epoch: 9 [0/50000 (0%)]	Loss: 0.123167
Train Epoch: 9 [128/50000 (0%)]	Loss: 0.160979
Train Epoch: 9 [256/50000 (1%)]	Loss: 0.15

Train Epoch: 9 [18816/50000 (38%)]	Loss: 0.140769
Train Epoch: 9 [18944/50000 (38%)]	Loss: 0.116762
Train Epoch: 9 [19072/50000 (38%)]	Loss: 0.180412
Train Epoch: 9 [19200/50000 (38%)]	Loss: 0.158794
Train Epoch: 9 [19328/50000 (39%)]	Loss: 0.079397
Train Epoch: 9 [19456/50000 (39%)]	Loss: 0.123206
Train Epoch: 9 [19584/50000 (39%)]	Loss: 0.147400
Train Epoch: 9 [19712/50000 (39%)]	Loss: 0.176753
Train Epoch: 9 [19840/50000 (40%)]	Loss: 0.170477
Train Epoch: 9 [19968/50000 (40%)]	Loss: 0.116439
Train Epoch: 9 [20096/50000 (40%)]	Loss: 0.176422
Train Epoch: 9 [20224/50000 (41%)]	Loss: 0.208891
Train Epoch: 9 [20352/50000 (41%)]	Loss: 0.184624
Train Epoch: 9 [20480/50000 (41%)]	Loss: 0.142001
Train Epoch: 9 [20608/50000 (41%)]	Loss: 0.200574
Train Epoch: 9 [20736/50000 (42%)]	Loss: 0.145192
Train Epoch: 9 [20864/50000 (42%)]	Loss: 0.143252
Train Epoch: 9 [20992/50000 (42%)]	Loss: 0.113440
Train Epoch: 9 [21120/50000 (42%)]	Loss: 0.169238
Train Epoch: 9 [21248/50000 (43%)]	Loss: 0.195504


Train Epoch: 9 [39808/50000 (80%)]	Loss: 0.100130
Train Epoch: 9 [39936/50000 (80%)]	Loss: 0.159748
Train Epoch: 9 [40064/50000 (80%)]	Loss: 0.189636
Train Epoch: 9 [40192/50000 (81%)]	Loss: 0.105372
Train Epoch: 9 [40320/50000 (81%)]	Loss: 0.146743
Train Epoch: 9 [40448/50000 (81%)]	Loss: 0.135434
Train Epoch: 9 [40576/50000 (81%)]	Loss: 0.129605
Train Epoch: 9 [40704/50000 (82%)]	Loss: 0.147148
Train Epoch: 9 [40832/50000 (82%)]	Loss: 0.201103
Train Epoch: 9 [40960/50000 (82%)]	Loss: 0.153559
Train Epoch: 9 [41088/50000 (82%)]	Loss: 0.143042
Train Epoch: 9 [41216/50000 (83%)]	Loss: 0.130791
Train Epoch: 9 [41344/50000 (83%)]	Loss: 0.111665
Train Epoch: 9 [41472/50000 (83%)]	Loss: 0.156128
Train Epoch: 9 [41600/50000 (83%)]	Loss: 0.134231
Train Epoch: 9 [41728/50000 (84%)]	Loss: 0.157011
Train Epoch: 9 [41856/50000 (84%)]	Loss: 0.191724
Train Epoch: 9 [41984/50000 (84%)]	Loss: 0.130107
Train Epoch: 9 [42112/50000 (84%)]	Loss: 0.130936
Train Epoch: 9 [42240/50000 (85%)]	Loss: 0.147888


Train Epoch: 10 [10368/50000 (21%)]	Loss: 0.074723
Train Epoch: 10 [10496/50000 (21%)]	Loss: 0.097960
Train Epoch: 10 [10624/50000 (21%)]	Loss: 0.121322
Train Epoch: 10 [10752/50000 (22%)]	Loss: 0.096477
Train Epoch: 10 [10880/50000 (22%)]	Loss: 0.163041
Train Epoch: 10 [11008/50000 (22%)]	Loss: 0.101453
Train Epoch: 10 [11136/50000 (22%)]	Loss: 0.118060
Train Epoch: 10 [11264/50000 (23%)]	Loss: 0.113817
Train Epoch: 10 [11392/50000 (23%)]	Loss: 0.086307
Train Epoch: 10 [11520/50000 (23%)]	Loss: 0.134533
Train Epoch: 10 [11648/50000 (23%)]	Loss: 0.118312
Train Epoch: 10 [11776/50000 (24%)]	Loss: 0.170679
Train Epoch: 10 [11904/50000 (24%)]	Loss: 0.095101
Train Epoch: 10 [12032/50000 (24%)]	Loss: 0.137236
Train Epoch: 10 [12160/50000 (24%)]	Loss: 0.142296
Train Epoch: 10 [12288/50000 (25%)]	Loss: 0.155056
Train Epoch: 10 [12416/50000 (25%)]	Loss: 0.156571
Train Epoch: 10 [12544/50000 (25%)]	Loss: 0.154248
Train Epoch: 10 [12672/50000 (25%)]	Loss: 0.101148
Train Epoch: 10 [12800/50000 (2

Train Epoch: 10 [30976/50000 (62%)]	Loss: 0.111719
Train Epoch: 10 [31104/50000 (62%)]	Loss: 0.104721
Train Epoch: 10 [31232/50000 (63%)]	Loss: 0.096948
Train Epoch: 10 [31360/50000 (63%)]	Loss: 0.103888
Train Epoch: 10 [31488/50000 (63%)]	Loss: 0.141127
Train Epoch: 10 [31616/50000 (63%)]	Loss: 0.115702
Train Epoch: 10 [31744/50000 (64%)]	Loss: 0.169381
Train Epoch: 10 [31872/50000 (64%)]	Loss: 0.140411
Train Epoch: 10 [32000/50000 (64%)]	Loss: 0.171381
Train Epoch: 10 [32128/50000 (64%)]	Loss: 0.151579
Train Epoch: 10 [32256/50000 (65%)]	Loss: 0.161709
Train Epoch: 10 [32384/50000 (65%)]	Loss: 0.115055
Train Epoch: 10 [32512/50000 (65%)]	Loss: 0.140615
Train Epoch: 10 [32640/50000 (65%)]	Loss: 0.124322
Train Epoch: 10 [32768/50000 (66%)]	Loss: 0.136862
Train Epoch: 10 [32896/50000 (66%)]	Loss: 0.116984
Train Epoch: 10 [33024/50000 (66%)]	Loss: 0.110575
Train Epoch: 10 [33152/50000 (66%)]	Loss: 0.101478
Train Epoch: 10 [33280/50000 (67%)]	Loss: 0.152401
Train Epoch: 10 [33408/50000 (6

# TRAIN MODEL USING MART

In [39]:
#TBD

# TRAIN MODEL USING MMA

In [40]:
#TBD

# LOAD ALL PRE-TRAINED MODELS

In [41]:
TrainSGD = TrainSGD*True
TrainESGD = TrainESGD*True
TrainL2 = TrainL2*True
TrainLInf = TrainLInf*True
TrainSAT2 = TrainSAT2*True
TrainSATInf = TrainSATInf*True
TrainTRADES = TrainTRADES*True
TrainTRADESInf = TrainTRADESInf*True
TrainMART = TrainMART*False
TrainMMA = TrainMMA*False
TrainMMAInf = TrainMMAInf*False

In [42]:
TrainTRADESInf = True

In [44]:
# Load all the pre-trained models
if not TrainSGD:
    model_SGD = torch.load('../trainedmodels/'+dataset+'/SGD_ep10_lr0.1.pt').to(DEVICE)
if not TrainESGD:    
    model_ESGD = torch.load('../trainedmodels/'+dataset+'/ESGD_ep5_lr0.1.pt').to(DEVICE)
if not TrainL2:
    adv_model_linf = torch.load('../trainedmodels/'+dataset+'/AT2_ep2_lr0.1.pt').to(DEVICE)
if not TrainLInf:
    adv_model_l2 = torch.load('../trainedmodels/'+dataset+'/ATInf_ep2_lr0.1.pt').to(DEVICE)
if not TrainSAT2:
    model_SAT2 = torch.load('../trainedmodels/'+dataset+'/SAT2_ep2_lr0.1.pt').to(DEVICE)
if not TrainSATInf:
    model_SATInf = torch.load('../trainedmodels/'+dataset+'/SATInf_ep2_lr0.1.pt').to(DEVICE)
if not TrainTRADES:
    model_TRADES = torch.load('../trainedmodels/'+dataset+'/TRADES_ep2_lr0.1.pt').to(DEVICE)
if not TrainTRADESInf:
    model_TRADES = torch.load('../trainedmodels/'+dataset+'/TRADESInf_ep2_lr0.1.pt').to(DEVICE)    
'''
if not TrainMART:
    if dataset=='CIFAR10':
        model_MART = Net()
        if NetName == 'ResNet18':
            dics = torch.load('../trainedmodels/'+dataset+'/ResNet18_model_MART.pt')
        if NetName == 'WideResNet':
            dics = torch.load('../trainedmodels/'+dataset+'/rob_cifar_mart.pt')            
        model_MART.load_state_dict(dics)
        model_MART = model_MART.to(DEVICE)
if not TrainMMA:
    model_MMA = torch.load('../trainedmodels/'+dataset+'/'+dataset.lower()+'-L2-MMA-4.0-sd0/model_best.pt')
    model_MMAInf = torch.load('../trainedmodels/'+dataset+'/'+dataset.lower()+'-Linf-MMA-0.45-sd0/model_best.pt')
'''

"\nif not TrainMART:\n    if dataset=='CIFAR10':\n        model_MART = Net()\n        if NetName == 'ResNet18':\n            dics = torch.load('../trainedmodels/'+dataset+'/ResNet18_model_MART.pt')\n        if NetName == 'WideResNet':\n            dics = torch.load('../trainedmodels/'+dataset+'/rob_cifar_mart.pt')            \n        model_MART.load_state_dict(dics)\n        model_MART = model_MART.to(DEVICE)\nif not TrainMMA:\n    model_MMA = torch.load('../trainedmodels/'+dataset+'/'+dataset.lower()+'-L2-MMA-4.0-sd0/model_best.pt')\n    model_MMAInf = torch.load('../trainedmodels/'+dataset+'/'+dataset.lower()+'-Linf-MMA-0.45-sd0/model_best.pt')\n"

## Quantifying adversarial accuracy

In [45]:
def infnorm(x):
    infn = torch.max(torch.abs(x.detach().cpu()))
    return infn

In [46]:
def evaluate_against_adversary(model, k, eps, step, norm):
    total = 0
    acc = 0
    for x, y in val_loader:
        total += x.size(0)
        
        if norm == 2:
            x_adv = pgd(
                model, x.to(DEVICE), y.to(DEVICE), torch.nn.CrossEntropyLoss(), k=k, step=step, eps=eps, norm=2)
            print('rel. l2-norm of x_adv-x:',torch.norm(x_adv.detach().cpu()-x)/np.sqrt(x.size(0)))#/torch.norm(x))
        elif norm == 'inf':
            x_adv = iterated_fgsm(
                model, x.to(DEVICE), y.to(DEVICE), torch.nn.CrossEntropyLoss(), k=k, step=step, eps=eps, norm='inf')
            print('rel. linf-norm of x_adv-x:',infnorm(x_adv.detach().cpu()-x)/infnorm(x))
        y_pred = model(x_adv)

        acc += olympic.metrics.accuracy(y.to(DEVICE), y_pred) * x.size(0)

    return acc/total

## Evaluate robust models

In [47]:
loadResults = False

In [48]:
t1 = time.time()
if not loadResults:
    pgd_attack_range = [3.0] #np.arange(0, 6.1, 1./3)
    acc_SGD = []
    acc_ESGD = []
    acc_l2 = []
    acc_SAT2 = []
    acc_TRADES = []
    acc_MMA = []
    for eps in pgd_attack_range:
        print('eps:',eps)
        print('evaluating SGD network...')
        acc_SGD.append(evaluate_against_adversary(model_SGD, k=20, eps=eps, step=0.5, norm=2))
        print('evaluating ESGD network...')
        acc_ESGD.append(evaluate_against_adversary(model_ESGD, k=20, eps=eps, step=0.5, norm=2))
        print('evaluating SAT2 network...')
        acc_SAT2.append(evaluate_against_adversary(model_SAT2, k=30, eps=eps, step=0.3, norm=2))
        print('evaluating l2 network...')
        acc_l2.append(evaluate_against_adversary(adv_model_l2, k=30, eps=eps, step=1.5, norm=2))
        print('evaluating TRADES network...')
        acc_TRADES.append(evaluate_against_adversary(model_TRADES, k=30, eps=eps, step=1.5, norm=2))  
        #print('evaluating MART network...')
        #acc_MMA.append(evaluate_against_adversary(model_MMA, k=30, eps=eps, step=1.5, norm=2))          
print("time elapsed:",time.time()-t1)    

eps: 3.0
evaluating SGD network...
rel. l2-norm of x_adv-x: tensor(2.9203)
rel. l2-norm of x_adv-x: tensor(2.9299)
rel. l2-norm of x_adv-x: tensor(2.9110)
rel. l2-norm of x_adv-x: tensor(2.9079)
rel. l2-norm of x_adv-x: tensor(2.9165)
rel. l2-norm of x_adv-x: tensor(2.9116)
rel. l2-norm of x_adv-x: tensor(2.8897)
rel. l2-norm of x_adv-x: tensor(2.9204)
rel. l2-norm of x_adv-x: tensor(2.9440)
rel. l2-norm of x_adv-x: tensor(2.9237)
rel. l2-norm of x_adv-x: tensor(2.9232)
rel. l2-norm of x_adv-x: tensor(2.9266)
rel. l2-norm of x_adv-x: tensor(2.9064)
rel. l2-norm of x_adv-x: tensor(2.9315)
rel. l2-norm of x_adv-x: tensor(2.9236)
rel. l2-norm of x_adv-x: tensor(2.8902)
rel. l2-norm of x_adv-x: tensor(2.9205)
rel. l2-norm of x_adv-x: tensor(2.9212)
rel. l2-norm of x_adv-x: tensor(2.9150)
rel. l2-norm of x_adv-x: tensor(2.9059)
rel. l2-norm of x_adv-x: tensor(2.8753)
rel. l2-norm of x_adv-x: tensor(2.9225)
rel. l2-norm of x_adv-x: tensor(2.9158)
rel. l2-norm of x_adv-x: tensor(2.9100)
rel. 

rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)
rel. l2-norm of x_adv-x: tensor(2.9999)


In [49]:
if not loadResults:    
    accData = [acc_SGD,acc_ESGD,acc_l2,acc_SAT2,acc_TRADES,acc_MMA]
    np.save('../results/accData_l2.npy',accData)

In [ ]:
if loadResults:
    pgd_attack_range = [3.0] #np.arange(0.0, 6.1, 1./3)
    accData2 = np.load('../results/accData_l2.npy')
    [acc_SGD,acc_ESGD,acc_l2,acc_SAT2,acc_TRADES,acc_MMA] = accData2

In [51]:
with plt.style.context('ggplot'):
    COLOURS = plt.rcParams['axes.prop_cycle'].by_key()['color']
    #MARKERS = plt.rcParams['axes.prop_cycle'].by_key()['marker']

In [55]:
if len(pgd_attack_range) > 1:
    with plt.style.context('ggplot'):
        plt.rcParams["font.weight"] = "bold"
        plt.rcParams["axes.labelweight"] = "bold"
        plt.rcParams["axes.labelcolor"] = "black"
        fig, axes = plt.subplots(1, 1, figsize=(15,10))
        plt.figure(figsize=(14,7))
        axes.set_title('$L^2$-bounded adversary')
        axes.plot(pgd_attack_range, acc_SGD, label='SGD')
        axes.plot(pgd_attack_range, acc_ESGD, label='Entropy-SGD')
        axes.plot(pgd_attack_range, acc_SAT2, label='Data-Entropy-SGD ($L_2$)')
        axes.plot(pgd_attack_range, acc_l2, label='$L2$ training')
        axes.plot(pgd_attack_range, acc_TRADES, label='TRADES training')
        axes.plot(pgd_attack_range, acc_MMA, label='MMA training')

        axes.vlines([3], 0, 1, colors=COLOURS[1], linestyle='--')
        axes.set_ylabel('Accuracy')
        axes.set_xlabel('Epsilon')
        axes.set_ylim((0,1))
        axes.legend()
else:
    print(accData)

[[0.0], [0.0], [0.031149839743589744], [0.0], [0.00010016025641025641], []]


In [56]:
loadResultsFGSM = False

In [63]:
t1 = time.time()
if not loadResultsFGSM:
    fgsm_attack_range = [8/255] #np.arange(0.0, 0.52, 0.025)
    fgsm_acc_linf = []
    fgsm_acc_SGD = []
    fgsm_acc_ESGD = []
    fgsm_acc_SATInf = []
    fgsm_acc_TRADESInf = []
    fgsm_acc_MART = []
    fgsm_acc_MMAInf = []
    for eps in fgsm_attack_range:
        print('eps:',eps)
        print('evaluating SGD network...')
        fgsm_acc_SGD.append(evaluate_against_adversary(model_SGD, k=20, eps=eps, step=0.1, norm='inf'))
        print('evaluating ESGD network...')
        fgsm_acc_ESGD.append(evaluate_against_adversary(model_ESGD, k=20, eps=eps, step=0.1, norm='inf'))
        print('evaluating SAT2 network...')
        fgsm_acc_SATInf.append(evaluate_against_adversary(model_SATInf, k=10, eps=eps, step=0.1, norm='inf'))        
        print('evaluating linf network...')
        fgsm_acc_linf.append(evaluate_against_adversary(adv_model_linf, k=50, eps=eps, step=0.02, norm='inf'))
        print('evaluating TRADES network...')
        fgsm_acc_TRADESInf.append(evaluate_against_adversary(model_TRADESInf, k=50, eps=eps, step=0.02, norm='inf'))
        #print('evaluating MART network...')
        #fgsm_acc_MART.append(evaluate_against_adversary(model_MART, k=50, eps=eps, step=0.02, norm='inf'))
        #print('evaluating MMA network...')
        #fgsm_acc_MMAInf.append(evaluate_against_adversary(model_MMAInf, k=50, eps=eps, step=0.02, norm='inf'))
         
print("time elapsed:",time.time()-t1)    

eps: 0.03137254901960784
evaluating SGD network...
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x

rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(

rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
time elapsed: 632.8202047348022


In [64]:
if not loadResultsFGSM:    
    fgsmaccData = [fgsm_acc_SGD,fgsm_acc_ESGD,fgsm_acc_linf,fgsm_acc_SATInf, fgsm_acc_TRADESInf, fgsm_acc_MART, fgsm_acc_MMAInf]
    np.save('../results/fgsmaccData.npy',fgsmaccData)

In [ ]:
if loadResultsFGSM:
    fgsm_attack_range = [8/255] #np.arange(0.0, 0.52, 0.025)
    accData2 = np.load('../results/fgsmaccData.npy')
    [fgsm_acc_SGD,fgsm_acc_ESGD,fgsm_acc_linf,fgsm_acc_SATInf, fgsm_acc_TRADESInf, fgsm_acc_MART, fgsm_acc_MMAInf] = accData2

In [82]:
if len(fgsm_attack_range) > 1:
    with plt.style.context('ggplot'):
        plt.rcParams["font.weight"] = "bold"
        plt.rcParams["axes.labelweight"] = "bold"
        plt.rcParams["axes.labelcolor"] = "black"
        fig, axes = plt.subplots(1, 1, figsize=(15,10))
        plt.figure(figsize=(14,7))
        axes.set_title('$L^\infty$-bounded adversary')
        axes.plot(fgsm_attack_range, fgsm_acc_SGD, label='SGD')
        axes.plot(fgsm_attack_range, fgsm_acc_ESGD, label='Entropy-SGD')
        axes.plot(fgsm_attack_range, fgsm_acc_SATInf, label='Data-Entropy-SGD ($L_\infty$)')    
        axes.plot(fgsm_attack_range, fgsm_acc_linf, label='$L{\infty}$ training')
        axes.plot(fgsm_attack_range, fgsm_acc_TRADES, label='TRADES training')
        axes.vlines([0.25], 0, 1, colors=COLOURS[1], linestyle='--')
        axes.set_ylabel('Accuracy')
        axes.set_xlabel('Epsilon')
        axes.set_ylim((0,1))
        axes.legend()
else:
    print(fgsmaccData)

[[0.0026041666666666665], [0.039162660256410256], [0.34385016025641024], [0.17377804487179488], [0.055789262820512824], [], []]


In [96]:
tempFGSMSATInf_test1 = []
tempFGSMSATInf_test1.append(evaluate_against_adversary(model_SATInf_test1, k=10, eps=eps, step=0.1, norm='inf'))

rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(0.0314)
rel. linf-norm of x_adv-x: tensor(

In [97]:
tempFGSMSATInf_test1

[0.06300080128205128]